In [8]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [9]:
import numpy as np
import pandas as pd
import scipy

# Instansiate the Plotly charting library.
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px

# We use plotly.offline as this allows us to create interactive 
# visualisations without the use of an internet connection, 
# making our notebook more distributable to others. 
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

# The Cufflinks library allows us to directly bind 
# Pandas dataframes to Plotly charts. 
import cufflinks as cf
# Once again we use the Cufflinks library in offline mode. 
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options. We use these to make our interactive 
# visualisations more aesthetically appealing. 
from IPython.core.display import HTML
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from ipywidgets import interact, interact_manual, widgets

In [10]:
df = pd.read_csv('data/freeman_well_4_eng.csv')

In [11]:
df.head()

,Depth,GR,Log_ILD,DT,RHOB,NPHI,PHI,PERM,Facies,velocity,Facies_code
0,7682.5,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0
1,7683.0,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0
2,7683.5,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0
3,7684.0,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0
4,7684.5,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0


In [12]:
df.describe()

,Depth,GR,Log_ILD,DT,RHOB,NPHI,PHI,PERM,Facies,velocity,Facies_code
count,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7.286000e+03,7286.000000,7.286000e+03,7286.000000
mean,9517.276352,108.163436,16.647403,117.850574,2.330201,0.423016,0.206324,9.691201e+03,2.820615,9.447880e+05,1.820615
std,1056.427949,22.303899,167.046140,11.625088,0.081099,0.065935,0.052322,2.375493e+05,0.564634,4.050703e+05,0.564634
min,7682.500000,37.810800,0.081200,84.811200,1.863100,0.099500,0.055100,1.076985e-02,1.000000,5.128205e+02,0.000000
25%,8608.125000,101.914675,0.877900,108.402500,2.282625,0.384900,0.166700,1.836431e+00,3.000000,7.322923e+05,2.000000
50%,9518.750000,112.481050,1.025900,114.944650,2.347100,0.422100,0.195400,6.885164e+00,3.000000,9.747539e+05,2.000000
75%,10430.875000,120.165375,1.365575,127.965500,2.391600,0.461475,0.237000,4.675485e+01,3.000000,1.139082e+06,2.000000
max,11357.000000,178.320800,1950.000000,158.538400,2.564600,0.656900,0.507700,1.211313e+07,3.000000,1.231527e+07,2.000000


In [13]:
df.isnull().sum()

Depth          0
GR             0
Log_ILD        0
DT             0
RHOB           0
NPHI           0
PHI            0
PERM           0
Facies         0
velocity       0
Facies_code    0
dtype: int64

In [14]:
@interact
def correlations(column1=list(df.select_dtypes('number').columns), 
                 column2=list(df.select_dtypes('number').columns)):
    print(f"Correlation: {df[column1].corr(df[column2])}")

interactive(children=(Dropdown(description='column1', options=('Depth', 'GR', 'Log_ILD', 'DT', 'RHOB', 'NPHI',…

In [15]:
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:]):
    if x == y:
        print(f"Please select seperate variables for X and Y")
    else:
        df.iplot(kind='scatter', x=x, y=y, mode='markers', 
                 xTitle=x.title(), yTitle=y.title(), title=f'{y.title()} vs {x.title()}')
        ## if you are using Google Colab, comment out the above line of code and uncomment the lines below
        #fig = px.scatter(df, x=x, y=y, title=f'{y.title()} vs {x.title()}')
        #fig.show(renderer="colab")

interactive(children=(Dropdown(description='x', options=('Depth', 'GR', 'Log_ILD', 'DT', 'RHOB', 'NPHI', 'PHI'…

In [16]:
cscales = ['Greys', 'YlGnBu', 'Greens', 'YlOrRd', 'Bluered', 'RdBu',
            'Reds', 'Blues', 'Picnic', 'Rainbow', 'Portland', 'Jet',
            'Hot', 'Blackbody', 'Earth', 'Electric', 'Viridis', 'Cividis']

# We use the Figure Factory module of Plotly, which
# defines many unique and powerful plots to be used
# in Python. 
# For more info, see: https://plot.ly/python/figure-factory-subplots/
import plotly.figure_factory as ff

corrs = df.corr()

@interact
def plot_corrs(colorscale=cscales):
    figure = ff.create_annotated_heatmap(z = corrs.round(2).values, 
                                     x =list(corrs.columns), 
                                     y=list(corrs.index), 
                                     colorscale=colorscale,
                                     annotation_text=corrs.round(2).values)
    iplot(figure)
    ## if you are using Google Colab, comment out the above line of code and uncomment the line below
    #figure.show(renderer="colab")

interactive(children=(Dropdown(description='colorscale', options=('Greys', 'YlGnBu', 'Greens', 'YlOrRd', 'Blue…

In [17]:
@interact_manual
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:],
                 theme=list(cf.themes.THEMES.keys()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    
    if x == y:
        print(f"Please select seperate variables for X and Y")
    else:
        df.iplot(kind='scatter', x=x, y=y, mode='markers', 
                 xTitle=x.title(), yTitle=y.title(), 
                 text='Depth',
                 title=f'{y.title()} vs {x.title()}',
                theme=theme, colorscale=colorscale)
        ## if you are using Google Colab, comment out the above line of code and uncomment the line below
        #fig = px.scatter(df, x=x, y=y, title=f'{y.title()} vs {x.title()}')
        #fig.show(renderer="colab")

interactive(children=(Dropdown(description='x', options=('Depth', 'GR', 'Log_ILD', 'DT', 'RHOB', 'NPHI', 'PHI'…

In [20]:
from pore_pressure import eaton_method

In [21]:
from class2 import vs_from_vp
df['vs'] = vs_from_vp(df['velocity'])

KeyError: 'vp'

In [ ]:
df['Pore_pressure'] = eaton_method(df['velocity'], df['vs'], df['PHI'])

In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
# Define input features and target variable
X = final[['Log_ILD', 'RHOB', 'velocity', 'GR', 'Facies', 'Depth']]
y = final['PERM']

In [19]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define dictionary of models and their hyperparameters
models = {
    'svm': {
        'model': make_pipeline(StandardScaler(), SVR()),
        'params': {
            'svr__kernel': ['linear', 'rbf', 'poly'],
            'svr__C': [0.1, 1, 10],
            'svr__gamma': ['scale', 'auto']
        }
    },
    'random_forest': {
        'model': RandomForestRegressor(),
        'params': {
            'n_estimators': [50, 100, 150],
            'max_depth': [10, 20, 30]
        }
    },
    'decision_tree': {
        'model': DecisionTreeRegressor(),
        'params': {
            'max_depth': [5, 10, 15],
            'min_samples_split': [2, 5, 10]
        }
    },
    'neural_network': {
        'model': MLPRegressor(),
        'params': {
            'hidden_layer_sizes': [(10,), (20,), (30,)],
            'activation': ['relu', 'tanh', 'logistic']
        }
    }
}

NameError: name 'X' is not defined

In [ ]:
scores = []
# Loop over each model and perform grid search with cross-validation to find best hyperparameters 
## scoring='neg_mean_squared_error'
for model_name, model in models.items():
    clf = GridSearchCV(model['model'], model['params'], cv=5, n_jobs=-1, return_train_score=False)
    clf.fit(X_train, y_train)
    best_params = clf.best_params_
    
    # Evaluate best model on test set
    y_pred = clf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = clf.score(X_test, y_test)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': best_params,
        'RMSE': rmse,
        'R-squared': r2
    })
    
df = pd.DataFrame(scores,columns=['model', 'best_score', 'best_params', 'RMSE', 'R-squared'])
df

In [ ]:
import pickle

In [ ]:
with open('ppmodel_pickle', 'wb') as file:
    pickle.dump(model,file)